In [1]:
import pandas as pd

eur = pd.read_parquet("/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/EUR_USD.parquet")
eur

,open,high,low,close,volume,sma_20,sma_50,rsi,macd,macd_signal,...,bb_bandwidth,bb_percent,atr,plus_di,minus_di,adx,senkou_span_a,senkou_span_b,tenkan_sen,kijun_sen
timestamp,,,,,,,,,,,,,,,,,,,,,
2001-04-02 00:00:00+00:00,0.87520,0.87710,0.87520,0.87690,0.0,0.896980,0.912484,37.231329,-0.010675,-0.009232,...,6.778119,24.045356,0.009448,19.593174,30.045097,27.731717,0.921175,0.93025,0.886200,0.905250
2001-04-02 01:00:00+00:00,0.87690,0.87900,0.87690,0.87790,0.0,0.896980,0.912484,37.231329,-0.010675,-0.009232,...,6.778119,24.045356,0.009448,19.593174,30.045097,27.731717,0.921175,0.93025,0.886200,0.905250
2001-04-02 02:00:00+00:00,0.87790,0.87840,0.87750,0.87800,0.0,0.896980,0.912484,37.231329,-0.010675,-0.009232,...,6.778119,24.045356,0.009448,19.593174,30.045097,27.731717,0.921175,0.93025,0.886200,0.905250
2001-04-02 03:00:00+00:00,0.87800,0.87840,0.87710,0.87720,0.0,0.896980,0.912484,37.231329,-0.010675,-0.009232,...,6.778119,24.045356,0.009448,19.593174,30.045097,27.731717,0.921175,0.93025,0.886200,0.905250
2001-04-02 04:00:00+00:00,0.87720,0.87720,0.87600,0.87610,0.0,0.896980,0.912484,37.231329,-0.010675,-0.009232,...,6.778119,24.045356,0.009448,19.593174,30.045097,27.731717,0.921175,0.93025,0.886200,0.905250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-18 11:00:00+00:00,1.05481,1.05481,1.05325,1.05424,6483.0,1.073287,1.087723,32.149943,-0.009411,-0.007581,...,4.866687,18.129660,0.006578,12.544805,33.940502,43.558322,1.095150,1.10110,1.064695,1.071685
2024-11-18 12:00:00+00:00,1.05426,1.05488,1.05346,1.05472,6296.0,1.073287,1.087723,32.149943,-0.009411,-0.007581,...,4.866687,18.129660,0.006578,12.544805,33.940502,43.558322,1.095150,1.10110,1.064695,1.071685
2024-11-18 13:00:00+00:00,1.05472,1.05688,1.05454,1.05678,8695.0,1.073287,1.087723,32.149943,-0.009411,-0.007581,...,4.866687,18.129660,0.006578,12.544805,33.940502,43.558322,1.095150,1.10110,1.064695,1.071685
